In [ ]:
import torch
import torchvision
import numpy as np
from PIL import Image
from UnimatchV2_LULC.model.semseg.dpt import DPT
from utils.utils import make_patches, unpatchify, decode_segmap, LABELS, COLORS

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = DPT(
    **{'encoder_size': 'base', 'features': 128, 'out_channels': [96, 192, 384, 768],
       'nclass': 6})
model = torch.nn.SyncBatchNorm.convert_sync_batchnorm(model)
model.to(device)

unimatch_path = '/opt/models/exp/unimatchv2_0.pth'
checkpoint = torch.load(
    unimatch_path, map_location='cpu', weights_only=False)
new_state_dict = {}
for k, v in checkpoint['model'].items():
    new_key = k.replace('module.', '')
    new_state_dict[new_key] = v
model.load_state_dict(new_state_dict)


